# Documentation for Hydra Multi-Asset Verification Test
ACTION_LIST = [['test_add', 'test_q_for_r', 'test_r_for_r','test_r_for_q', 'test_remove']]



## Model Description

The methodology for the tests is to create two universes - one for Hydra and one for Uniswap - to subsequently be able to represent sequences of actions on each side. As the Hydra system is unique in its design and has no counterpart yet for approprate comparisons the role of the benchmark will be fulfilled by assembling several Uniswaps together in a way that everything that can be done in Hydra can be accordingly reproduced. This allows next to compare the implications on both systems and ultimately assess the quantitative and qualitative differences of results and impacts on the Hydra system and its agents. 

This approach necessitates the definition and specification of two models:
   - 1 Hydra Omnipool model with one instance that is defined in line with the [Hydra Mathematical Specification](https://hackmd.io/M7OeWimITKGVxBDHGQa6gQ?view)
   - 1 Uniswap model with several instances that each are responsible for a particular mechanism
        - which of 1 instance is used for liquidity provision in asset $R_i$: this is modeled as a Uniswap of $R_i$ and $Q$
        - which of 1 instance is used for liquidity provision in asset $R_j$: this is modeled as a Uniswap of $R_j$ and $Q$
        - which of 1 instance is used for swaps between assets $R_i$ and $R_j$: this is modeled as a Uniswap of $R_i$ and $R_j$
        
![](https://i.imgur.com/yb6Zh3T.png)
        
## Test cases

A variety of testcases can be explored with this model resulting from a combination of the following parameters
   
   - Event: Add Liquidity, Remove Liquidity, Swap Asset, Swap Asset for Base
   - Asset: $R_i$, $R_j$, $R_k$, $Q$
   - Multiple Action Type: Alternating, $i$ only, $j$ only
   - Composite Action Type: Alternating, Trade Bias
   - Enable Symmetric Liquidity: True, False
    
In addition to these settings also a fee structure can be imposed:

   - Fixed Fee

## Test agents

The model specifies a number of eight agents that each have their local balances, can interact with the system and are modeled to be responsible for **one action only**. This allows to track for each testcase how the balances will develop and to show independently which events will trigger changes to local and global states. Most importantly directly and indirectly affected variables can be easily identified.

Two dataframes of agents are initialized identically for both the Uniswap and the Hydra world and then work their way through on each side according to the testcase. Afterwards the effects on both sides can be compared.

![](https://i.imgur.com/r9IAJMr.png)



## Composite Action Tests

Besides being able to test all mechanisms individually also compositions of actions can be performed. This means selecting and combining two from the above testcases and letting the respective agents take the actions repeadetly. A composition action test therefore is defined as two actions which each can be a combination of *event* and *asset* selected from the list of testcases above, for example:
- Add liquidity in $R_i$ and Add liquidity in $R_j$
- Add liquidity in $R_i$ and Swap $R_i$ for $Q$
- Remove liquidity in $R_j$ and Swap $R_i$ for $Q$
- ...

In total there would be 28 composit pairs which can be derived from this decision tree:
selecting any one of the four primitive mechanisms of: **trade in/out** or **add/remove liquidity** of some given asset and then the subsequent action could be the three remaining mechanisms of the same asset or doing anyone of the four mechanisms in another asset.  

These basic combinations can be explored before further proceeding with
- trade sizes,
- trade frequencies,
- randomization,...



# Testcase in this notebook


This notebook documents the results of the following experiment:
- **Action 1**: One agent adds liquidity in XX
- **Action 2**: Another agent trades XX for XX.

which is a reslt of the following settings:

- exo_trade = ['test_q_for_r'] # agent 0
- exo_liq = ['test_add'] # agent 3
- exo_asset = ['alternating']
- exo_composite = ['alternating'] 

## Introduction to Uniswap
>Uniswap is an automated market maker for exchanging ERC20 tokens. Anyone can become a liquidity provider, and invest in the liquidity pool of an ERC20 token. This allows other users to trade that token for other tokens at an exchange rate based on their relative availibility. When a token trade is executed, a small fee is paid to the liquidity providers that enabled the transaction.
https://uniswap.io/

There are basically eight ways (_mechanisms_) in which an agent can interact with an instance of Uniswap (https://github.com/Uniswap/uniswap-v1/blob/master/contracts/uniswap_exchange.vy):
1. `addLiquidity`: deposit ETH and tokens in the liquidity pool; mint UNI tokens - "shares" of that Uniswap instance - in exchange;
1. `removeLiquidity`: burn UNI tokens; withdraw a proportional amount of ETH and tokens in exchange;
1. `ethToTokenInput`: user specifies an exact amount of ETH they send; receives corresponding amount of tokens
1. `ethToTokenOutput`: user sends some ETH and specifies an exact amount of tokens they want to buy; Uniswap refunds ETH sent in excess
1. `tokenToEthInput`: user specifies an exact amount of tokens they send; receives corresponding amount of ETH
1. `tokenToEthOutput`: user specifies an exact amount of ETH they want to buy; Uniswap takes the corresponding amount of tokens from the user account
1. `tokenToTokenInput`: user specifies an exact amount of "token A" they send; receives corresponding amount of "token B" (effectively a `tokenToEthInput` in instance A combined with a `ethToTokenInput` in instance B)
1. `tokenToTokenOutput`: user specifies an exact amount of "token B˜ they want to buy; Uniswap takes the corresponding amount of "token A" from the user account (effectively a `ethToTokenOutput` in instance B combined with a `tokenToEthOutput` in instance A)

## Limitations and simplifications of this model
* When triggering mechanisms, Uniswap users may include conditions they would like to apply to the transaction. For example, they could define a minimum amount of UNI tokens they wish to receive as a result of adding liquidity to the pool; or the maximum number of tokens they are willing to pay for the amount of ETH requested in a `tokenToEthOutput` operation; or a deadline after which the transaction should not be processed. **This simplified model does not account for this kind conditioning.**

* The `ethToTokenOutput`, `tokenToEthOutput` mechanisms are not implemented. **All swaps are treated as of the "input defined" type.**

* The `tokenToToken` mechanism are not implemented, as they are merely a `tokenToEth` in the context of a single Uniswap instance.

* **User _behavior_ has not been modeled**. User _actions_ are derived from the history of events of the Uniswap instance being analyzed.

## Introduction to Hydra
>Hydra is an automated market maker that generalizes the concept of Uniswap and Balancer to a) an arbitrary number of dimensions and b) a variant weight distaribution of assets in the pool that changes in accordance with action sequences in the pool. Anyone can become a liquidity provider, and invest in the 'Omnipool'. This allows other users to trade that token for other tokens at an exchange rate based on their relative availibility. When a token trade is executed, a small fee is paid to the liquidity providers that enabled the transaction.
https://hydradx.io/

There is a predefined set of (_mechanisms_) in which an agent can interact with an the Hydra Protocol :
1. `addLiquidity`: deposit a risk asset in the omnipool; mint HDX tokens - "shares" of this risk asset - in exchange;
1. `removeLiquidity`: burn HDX tokens; withdraw a proportional amount of the risk asset in exchange;
1. `tokenToTokenInput`: user specifies an exact amount of "token A" they send; receives corresponding amount of "token B" 
1. `tokenToTokenOutput`: user specifies an exact amount of "token B˜ they want to buy; Hydra takes the corresponding amount of "token A" from the user account 



## Model File structure

The folder **model** contains:

 - a file config.py: 

    This file configures the simulation experiments by setting the number of monte carlo runs, the number of timesteps for the simulation and the simulation parameters. It also aggregates the partial states and the initial state variables.
    
    The purpose of this file is to *configure and initialize* the experiment.
    
    
 - a file partial_state_update_block.py

    This file defines each partial state update block individually. This feature allows cadCAD to divide each simulation timestep into several 'blocks' where different state variables are mutated conditioned upon the policy input.
    
    In the current setting 
    - the first block maps the user action to the appropriate mechanism in the uniswap world
    - the second block maps the user action to the appropriate mechanism in the hydra world
    - the third block resolves the quantities of HDX and weights in the hydra world
    - the forth block calculates the metrics: the swap prices in the uniswap world and the pool prices in the hydra world

    This file corresponds to the system specification diagram, where each column is one partial state update block 
    
    The purpose of this file is to *define* the sequence and details of state updates for each simulation timestep. The relevant mechanisms are imported from the files in the **parts** folder
    
    
 - a file plot_utils.py

    This file defines all plot functions that can be called in the jupyter notebook to visualize the results. In general there are three types of plots:
    - plots related to the uniswap world (do not contain the word 'hydra')
    - plots related to the hydra world (contain the word 'hydra')
    - general plots applicable to both worlds (f.e Impermanent Loss)

    It also can be distinguished between plots related to global system variables and local agent variables:
    - global variables (do not contain the word 'agent')
    - local variables (contain the word 'agent')

    The purpose of this file is to *define* the way how particular properties can be plotted over time.
    

 - a file run.py

    This file *defines* the run and postprocessing methods and is used to execute the simulation run:
    - run() is used to execute the simulation and create a dataframe
    - postprocessing is used to extract relevant metrics from the simulation output in the dataframe

     The purpose of this file is to *execute* the simulation and *create* metrics.
     
     
 - a file state_variables.py

    This file *sets* the number of agents and *defines* the local (agent-level) state variables, *sets* the number of assets and *defines* the global (system-state) variables of the system, *computes* the prices of the assets and *defines* the initial state object
    - the agent states describe their holdings of assets that agents have contributed to the pool or still hold outside as well as their quantities of shares they received for their contribution
    - the global states describe the quantities of assets in the system
    - price variables are computed from the states both in the uniswap and hydra world
    - the initial state object consists of 
        - two agent dataframes, one for each world respectively
        - two sets of global variables, one for each world respectively
        - prices for each world (? is pool = prices ?)

    The purpose of this file is to *define* the global and local states of the system.
    
    
 - a file sys_params.py

    This file *contains* the system parameters and allows to select items from a list for them. Following parameters can be defined:
    - trade action type
    - liquidity action type
    - multiple asset action type
    - composite action type
    - liquidity add type (symmetry)

    In addition the following is set
    - initial values for assets 
    - hard coded fee value
    - choice of the system parameters from the list above
    

 - a folder **parts** which itself contains general files relevant for both instances, files related specifically to Uniswap and files related specifically to Hydra 
      - [action.py](model/parts/action.py)
       This is an action dictionary which drives the model. As there are no behavioral assumptions in the model yet, currently actions are prescribed actions in testing mode derived from events. However, action schema respects https://www.kaggle.com/markusbkoch/uniswap-ethdai-exchange as much as possible, allowing for future analysis against uniswap transaction datasets.
        This file contains a function called 'actionDecoder' which takes its inputs from the file sys_params.py and maps those encoded events to agent actions. At first an arbitrary action dictionary is defined which then is adapted according to the choices of system parameters. In particular relevant are the parameter definitions of the asset, the composite and whether there is a trade or liquidity event defined.
        
    - [uniswap.py](model/parts/uniswap.py)
     Reads in action output and directs the action to the appropriate Uniswap mechanism for each corresponding Uniswap instance. Currently there are two instances in place, namely the R_i<>Q pair and the R_j<>Q pair. Both instances have their respective requisite state variables: UNI_Qx, UNI_Rx, and UNI_Sx.

    - [utils.py](model/parts/utils.py)
    Computes the state update for each Uniswap state variable and outputs the updated global state variables.
        
     - [agent_utils.py](model/parts/agent_utils.py)
     Computes the corresponding agent local state for each Uniswap agent resolving their action and outputs the updates local state variables.

      -  [hydra.py](model/prts/hydra.py)
     Reads in action output and directs the action to the appropriate Hydra mechanism for each state variable: Q, Sq, and pool.
    
      - [hydra_utils.py](model/parts/hydra_utils.py)
     Computes the state update for each Hydra state variable

    -  [asset_utils.py](model/parts/asset_utils.py)
     Computes the state update for each Reserve Asset in a Hydra Omnipool

    -  [hydra_agent_utils_class.py](model/parts/hydra_agent_utils_class.py)
     Computes the corresponding agent local state for each Hydra agent resolving their action and outputs the updates local state variables.
     
     #### The interrelations between the files are shown here:
     
    ![](https://i.imgur.com/mNhfQE4.png)



# Testcase Definition

#### Block 1:

| #   |   Run   | a  | $\mu_i$ |          $\sigma_i$          | $\mu_j$ |          $\sigma_j$          | $\mu$ | $\sigma$ | Total runs         |
| --- |:-------:|:-------:|:-------:|:----------------------------:|:-------:|:----------------------------:| ----- | -------- | ------------------ |
| 1   | 001-100 |   0.5   |   $L$   | $\frac{\mu_i}{\sigma_i}= 2$  |   $L$   | $\frac{\mu_j}{\sigma_j}= 2$  | $1.000$ | $0$      | $100$ |
| 2   | 101-200 |   0.5   |   $L$   | $\frac{\mu_i}{\sigma_i}= 50$ |   $L$   | $\frac{\mu_j}{\sigma_j}= 50$ | $1.000$ | $0$      | $100$ |
| 3   | 201-300 |    1    |   $L$   | $\frac{\mu_i}{\sigma_i}= 2$  |   $L$   | $\frac{\mu_j}{\sigma_j}= 2$  | $1.000$ | $0$      | $100$ |
| 4   | 301-400 |    1    |   $L$   | $\frac{\mu_i}{\sigma_i}= 50$ |   $L$   | $\frac{\mu_j}{\sigma_j}= 50$ | $1.000$ | $0$      | $100$ |
| 5   | 401-500 |   1.5   |   $L$   | $\frac{\mu_i}{\sigma_i}= 2$  |   $L$   | $\frac{\mu_j}{\sigma_j}= 2$  | $1.000$ | $0$      | $100$ |
| 6   | 501-600 |   1.5   |   $L$   | $\frac{\mu_i}{\sigma_i}= 50$ |   $L$   | $\frac{\mu_j}{\sigma_j}= 50$ | $1.000$ | $0$      | $100$ |

All tests in block 1 are performed for **each combination** of $\mu_i$ and $\mu_j$ selectable from $L$ which results in $100$ runs; where 

$\mu_i \in L =\{10.000, 25.000, 50.000, 75.000, 100.000, 250.000, 500.000, 750.000, 1.000.000, 2.000.000 \}$

and

$\mu_j \in L =\{10.000, 25.000, 50.000, 75.000, 100.000, 250.000, 500.000, 750.000, 1.000.000, 2.000.000 \}$

### Goals for Friday:
- Having one notebook that runs block 1 in its entirety and presents the results as a general descriptive analysis

### Experiment for Friday:

I want to have:
- 600 runs through the model for experiments 1-6
- In these experiments there should be 
    - one agent (nr 2) adding liquidity and removing it afterwards
    - one agent (nr 5) randomly trading in between
- This runs should be performed for 
- 1 notebook that would show in fan plots how Uniswap & Hydra instances perform for these 600 runs
- select which runs would make sense for that

## Run Model

The following command lines import all packages, experiments and run the model 

This is the execution of cadcad. This writes the config object, does the execution and returns the rdf - which is a data object that shows the rows and columns of results in a dataframe

Here is one additional option as well. The postprocessing causes the index of the results to be every forth because of the cadcad architecture where the partial state updates are the substeps. With this option selected we see only the end of each timestep - the last substep. Therefore only final results are shown. To get a line for each substep the postprocessing needs to be deactivated. 

In [1]:
from model.config_wrapper import ConfigWrapper

from model import run_wrapper
from cadCAD import configs

# parametric_experiment = ConfigWrapper()
# get_keys = parametric_experiment.get_config()
# get_keys[0]['M']
# parametric_experiment.get_config()

running file: config_wrapper.py
end of file: config_wrapper.py
running file: run_wrapper.py
end of file: run_wrapper.py


In [2]:
import model as model


In [3]:
parametric_experiment = ConfigWrapper(model)


running file: state_variables.py
running file: initialize_liquidity.py
end of file: initialize_liquidity.py
running file: asset_utils.py
end of file: asset_utils.py
end of file: state_variables.py
running file: partial_state_update_block.py
running file: action_list.py
end of file: action_list.py
end of file: partial_state_update_block.py
running file: sim_setup.py
end of file: sim_setup.py
m_state_variables {'UNI_Qi': 20000, 'UNI_Ri': 59580, 'UNI_Si': 10000, 'UNI_Qj': 20000, 'UNI_Rj': 11824049, 'UNI_Sj': 10000, 'UNI_ij': 59580, 'UNI_ji': 11824049, 'UNI_Sij': 704476839420, 'uni_agents':    m  r_i_out  r_i_in       h     q_i     s_i  s_q  r_j_out  r_j_in     q_j  \
0  0   100000  110000  140000  170000  150000    0   120000  130000  180000   
1  1   100000  110000  140000  170000  150000    0   120000  130000  180000   
2  2   100000  110000  140000  170000  150000    0   120000  130000  180000   
3  3   100000  110000  140000  170000  150000    0   120000  130000  180000   
4  4   1000

In [4]:
del configs[:]

parametric_experiment.append()

NameError: name 'initial_state' is not defined

In [ ]:
get_keys = parametric_experiment.get_config()
get_keys[0]['M']

In [ ]:
# Dependences
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Experiments
from model import run
from model.parts.utils import *
from model.plot_utils import *

pd.options.display.float_format = '{:.2f}'.format

%matplotlib inline

# df = run.run()
# rdf = run.postprocessing(df)
# rdf = df # debugging substeps
# pd.set_option("mode.chained_assignment", None)


In [ ]:
(data, tensor_field, sessions) = run_wrapper.run(drop_midsteps=True)

experiments = data
experiments1 = data


# Subsets:

For the parameter sweeps there are subsets of the simulation that categorize the current test run.
The run number can be retrieved from the header of the cadCAD execution (see above)


In [ ]:
experiments.subset

In [ ]:
experiments.subset.unique()

In [ ]:
experiments = experiments.sort_values(by =['subset']).reset_index(drop=True)
len(experiments)

In [ ]:
experiments.head()

In [ ]:
experiments.tail().pool

In [ ]:
print(experiments.pool[experiments.first_valid_index()].pool['i'])

In [ ]:
def get_M(k, v):
    if k == 'sim_config':
        k, v = 'M', v['M']
    return k, v

config_ids = [
    dict(
        get_M(k, v) for k, v in config.__dict__.items() if k in ['simulation_id', 'run_id', 'sim_config', 'subset_id']
    ) for config in configs
]

# Effects of variables for different scenarios

The following function takes the scanarios that were run and creates a fan plot.

```
def param_fan_plot2(experiments, config_ids, swept_variable, y_variable, *args):
    """
    experiments is the simulation result dataframe.
    config_ids is the list configs executed upon in the simulation.
    swept_variable is the key (string) in config_ids that was being tested against.
    y_variable is the state_variable (string) to be plotted against default timestep.
    *args for plotting more state_variables (string).
    """
```

The y_variable in this case is UNI_i. 
On the y-axis we see therefore a fan plot that shows the **mean**, the **max** and the **min** of the y-variable for all runs.

```
        df.plot(x='timestep', y=('UNI_Ri','mean'),label = y_variable, ax=ax, legend=True)

        ax.fill_between(df.timestep, df[('UNI_Ri','min')], df[('UNI_Ri','max')], alpha=0.3) 
```

For UNI_i this is not impressive, as all mean, max and min are the same:


# Q - Ri instance

In [ ]:
model_history = experiments[['UNI_Qi', 'UNI_Ri', 'UNI_Si']]
model_history.columns = ['model_UNI_Qi', 'model_UNI_Ri', 'model_UNI_Si']

uniswap_j = experiments[['UNI_Qj', 'UNI_Rj', 'UNI_Sj']]
uniswap_j.columns = ['UNI_Qj', 'UNI_Rj', 'UNI_Sj']

uniswap_ij = experiments[['UNI_ij', 'UNI_ji', 'UNI_Sij']]
uniswap_ij.columns = ['UNI_ij', 'UNI_ji', 'UNI_Sij']

hydra = experiments[['Q', 'H', 'Sq']]
hydra.columns = ['Hydra_Q', 'Hydra_H', 'Hydra_Sq']

In [ ]:
plt.figure(figsize=(20,6))
ax = plt.subplot(131)
model_history.astype(float).plot(ax=ax, y=['model_UNI_Qi'])
#model_history.astype(float).plot(ax=ax, y=['UNI_Qi'])
plt.title('UNISWAP Asset i Q-Base Token')

ax = plt.subplot(132)
model_history.astype(float).plot(ax=ax, y=['model_UNI_Ri'])
plt.title('UNISWAP Asset i Reserve Risk')

ax = plt.subplot(133)
model_history.astype(float).plot(ax=ax, y=['model_UNI_Si'])
plt.title('UNISWAP Asset i Shares')
plt.show()

In [ ]:
param_fan_plot2(experiments, config_ids, 'a', 'UNI_Ri')

In [ ]:
param_fan_plot(experiments, config_ids, 'a', 'UNI_Si', 'UNI_Si')

In [ ]:
param_fan_plot(experiments, config_ids, 'a', 'UNI_Qi', 'UNI_Qi')

# Q - Rj instance

In [ ]:
param_fan_plot(experiments, config_ids, 'a', 'UNI_Rj', 'UNI_Rj')

In [ ]:
param_fan_plot(experiments, config_ids, 'a', 'UNI_Sj', 'UNI_Sj')

In [ ]:
param_fan_plot3(experiments, config_ids, 'a', 'UNI_Qj', 'UNI_Qj')

# ij - instance

In [ ]:
plt.figure(figsize=(20,6))
ax = plt.subplot(131)
uniswap_ij.astype(float).plot(ax=ax, y=['UNI_ij'])
plt.title('UNISWAP IJ Instance Asset i Token')

ax = plt.subplot(132)
uniswap_ij.astype(float).plot(ax=ax, y=['UNI_ji'])
plt.title('UNISWAP IJ Instance Asset j Token')

ax = plt.subplot(133)
uniswap_ij.astype(float).plot(ax=ax, y=['UNI_Sij'])
plt.title('UNISWAP IJ Instance Shares')
plt.show()

In [ ]:
param_fan_plot(experiments, config_ids, 'a', 'UNI_ij', 'UNI_ij')

In [ ]:
param_fan_plot3(experiments, config_ids, 'a', 'UNI_ij')

In [ ]:
param_fan_plot3(experiments, config_ids, 'a', 'Wq', 'Wq')

In [ ]:
param_fan_plot3(experiments, config_ids, 'a', 'UNI_Sij', 'UNI_Sij')

In [ ]:
results = pd.read_pickle ('C:/Users/paruc/Documents/Github/hydra/hydra_multi_class/experiment_block1_experiments20210429.pkl')
results

results is structured as [k][i][j] where
- k = 0 or 1
- i from 0 to 9
- j from 0 to 9

In [ ]:
#results[][][]

# Agents

## Uniswap

In [ ]:
agent_plot(experiments,'Uniswap Token Holding for Agent ', 200) #,4,'j')

In [ ]:
agent_value_plot(experiments,'Uniswap Value of S, R, and H in H equivalent', 200) #,2, 'i')

## Hydra

In [ ]:
hydra_agent_plot(experiments,'Hydra Token Holdings for Agent ', 200)

In [ ]:
hydra_agent_value_plot(experiments,'Hydra Omnipool Value of Si, Ri, and H in H equivalent', 200) #,2, 'i')

In [ ]:
IL_plot(experiments,'Impermanent Loss in Uniswap Instances Over Fixed Windows of ', 10)

In [ ]:
experiments['pool'].pool


In [ ]:
display(list(experiments.columns.values))
display(experiments.pool)


In [ ]:
experiments = experiments.sort_values(by =['subset']).reset_index(drop=True)

cols = 1
rows = 1
cc_idx = 0

while cc_idx<len(experiments):
    cc = experiments.iloc[cc_idx]['pool']
    cc_label = experiments.iloc[cc_idx]['pool']
    secondary_label = experiments.iloc[cc_idx]['UNI_Ri'] # what is the sdry label here?
    sub_experiments = experiments[experiments['pool']==cc]
    cc_idx += len(sub_experiments)
    fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(15*cols,7*rows))
    sub_experiments.reset_index(drop=False)
    for i, experiment in sub_experiments.iterrows():
        df = experiment['pool']
        df['R_i'] = df.pool(lambda x: np.array(x['i']['R']))
        df['S_i'] = df.pool(lambda x: np.array(x['i']['S']))
        df['W_i'] = df.pool(lambda x: np.array(x['i']['W']))
        df['P_i'] = df.pool(lambda x: np.array(x['i']['P']))

        df['R_i'] = np.array(df.R_i,dtype = float)
        df['S_i'] = np.array(df.S_i,dtype = float)
        df['W_i'] = np.array(df.W_i,dtype = float)
        df['P_i'] = np.array(df.P_i,dtype = float)
        
        df_j = df.groupby('timestep').agg({'j_wallet': ['min', 'mean', 'max']}).reset_index()
        df_r = df.groupby('timestep').agg({'r_wallet': ['min', 'mean', 'max']}).reset_index()
        df_p = df.groupby('timestep').agg({'p_wallet': ['min', 'mean', 'max']}).reset_index()
        df_k = df.groupby('timestep').agg({'k_wallet': ['min', 'mean', 'max']}).reset_index()

        plot_label = experiment['AB_Test']
        ax = axs
        title = 'Tokens by Roles' + '\n' + 'Scenario: ' + str(cc_label*100)  + ' % Route Allocation, ' + \
                str(secondary_label*100)  + ' % Store Allocation'
        ax.set_title(title)
        ax.set_ylabel('Tokens')
        colors = ['b','orange', 'g', 'r']

        ax.plot(df_j.timestep, df_j['j_wallet']['mean'],color = colors[0], label='j_wallet')
        ax.plot(df_r.timestep, df_r['r_wallet']['mean'],color = colors[1],label='r_wallet')
        ax.plot(df_p.timestep, df_p['p_wallet']['mean'],color = colors[2],label='p_wallet')
        ax.plot(df_k.timestep, df_k['k_wallet']['mean'],color = colors[3],label='k_wallet')

        ax.legend()

        ax.fill_between(df_j.timestep, df_j[('j_wallet','min')], df_j[('j_wallet','max')], alpha=0.3 ,color = colors[0],label='j_wallet')        
        ax.fill_between(df_r.timestep, df_r[('r_wallet','min')], df_r[('r_wallet','max')], alpha=0.3 ,color = colors[1],label='r_wallet')        
        ax.fill_between(df_p.timestep, df_p[('p_wallet','min')], df_p[('p_wallet','max')], alpha=0.3 ,color = colors[2],label='p_wallet')        
        ax.fill_between(df_k.timestep, df_k[('k_wallet','min')], df_k[('k_wallet','max')], alpha=0.3 ,color = colors[3],label='k_wallet')        
         
        ax.set_xlabel('Blocks')
        ax.grid(color='0.9', linestyle='-', linewidth=1)

        plt.tight_layout()
        
fig.tight_layout(rect=[0, 0, 1, .97])
fig.patch.set_alpha(1)
display(fig)
plt.close()

In [ ]:
pd.read_pickle ('C:/Users/paruc/Documents/Github/hydra/hydra_multi_class/experiment_block1_test_20210427.pkl')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

N = 1000
x = np.linspace(0, 10, N)
y = x**2
ones = np.ones(N)

vals = [30, 20, 10] # Values to iterate over and add/subtract from y.

fig, ax = plt.subplots()

for i, val in enumerate(vals):
    alpha = 0.5*(i+1)/len(vals) # Modify the alpha value for each iteration.
    ax.fill_between(x, y+ones*val, y-ones*val, color='red', alpha=alpha)

ax.plot(x, y, color='red') # Plot the original signal

plt.show()

In [ ]:
agent_plot(experiments,'Uniswap Token Holding for Agent ', 100) #,4,'j')

In [ ]:
hydra_agent_plot(experiments,'Hydra Token Holdings for Agent ', 100)

In [ ]:
experiments['hydra_agents'][400]

How to 
- extract particular data from the dataframe, conditioned on an attribute selection 
- sort data in this new dataframe

In [ ]:
experiments1.loc[experiments['subset'] == 1].sort_values(by =['index'])

How then further select particular entries from this dataframe:
- by defining colomn (?) and row (?)

In [ ]:
#experiments1.loc[experiments['subset'] == 1].sort_values(by =['index'])['index'][101]

How then further select entries by column and row number ?

In [ ]:
experiments1.loc[experiments['subset'] == 1].sort_values(by =['index']).iloc[0][2]

In [ ]:
np.linspace(0, 10, N)

In [ ]:
list(range(0, 5))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = list(range(0, 5))
y = experiments1.loc[experiments['subset'] == 1].sort_values(by =['index']).iloc[2][x]
fig, ax = plt.subplots()
ax.plot(x, y, color='red') # Plot the original signal

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

N = 1000
x = list(range(0, 5))
y = experiments1.loc[experiments['subset'] == 1].sort_values(by =['index']).iloc[0][x]
ones = np.ones(N)

vals = [30, 20, 10] # Values to iterate over and add/subtract from y.

fig, ax = plt.subplots()

for i, val in enumerate(vals):
    alpha = 0.5*(i+1)/len(vals) # Modify the alpha value for each iteration.
    ax.fill_between(x, y+ones*val, y-ones*val, color='red', alpha=alpha)

ax.plot(x, y, color='red') # Plot the original signal

plt.show()